<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def create_placeholders(input_H, input_W, no_channels, no_classes):

    X = tf.placeholder(tf.float32, shape=(None, input_H, input_W, no_channels), name="X")
    Y = tf.placeholder(tf.float32, shape=(None, no_classes), name="Y")
    
    return X, Y

In [ ]:
def initialize_parameters():
    
    W1 = tf.get_variable("W1", [4, 4, 3, 32], initializer = tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable("W2", [2, 2, 32, 32], initializer = tf.contrib.layers.xavier_initializer())

    parameters = {"W1": W1,
                  "W2": W2}
   
    return parameters

In [ ]:
def buildCNN(X):
  # list of parameters for the layers
  kernelVals = [5, 5, 3, 3, 3]
  featureVals = [1, 32, 64, 128, 128, 256]
  strideVals = poolVals = [(2, 2), (2, 2), (1, 2), (1, 2), (1, 2)]
  numLayers = len(strideVals)

        # create layers
  pool = cnn_input  # input to first CNN layer
  for i in range(numLayers+1):
    if i<=4:
      kernel = tf.Variable(tf.random.truncated_normal([kernelVals[i], kernelVals[i], featureVals[i], featureVals[i + 1]],
                                           stddev=0.1))
      conv = tf.nn.conv2d(input=pool, filters=kernel, padding='SAME', strides=(1,1,1, 1))
      relu = tf.nn.relu(conv)
      conv_norm = tf.compat.v1.layers.batch_normalization(relu, training=self.is_train)
      pool = tf.nn.max_pool2d(input=pool, ksize=( poolVals[i][0], poolVals[i][1]),
                                        strides=( 1,strideVals[i][0], strideVals[i][1],1), padding='VALID')
    else:
      flatten=tf.compat.v1.layers.flatten(pool)
      dense=tf.compat.v1.layers.dense(flatten)
      relu = tf.nn.relu(dense)
      dense_norm = tf.compat.v1.layers.batch_normalization(relu, training=self.is_train)
      output_dense=tf.compat.v1.layers.dense(dense_norm)
  return output_dense

      

   

In [ ]:
def compute_cost(Z3, Y):

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    return cost

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64):
    # number of training examples
    m = X.shape[0]                  
    mini_batches = []
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches